In [9]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [10]:
img_test = cv2.imread('motorbike.JPG')
cv2.imshow('img test',img_test)
cv2.waitKey()
cv2.destroyAllWindows()


In [4]:
def addnoise(pic):
    h,w = pic.shape[:2]  #获取图像的宽高信息
    nums = 5000
    rows = np.random.randint(0, h, (5000), dtype = int)
    cols = np.random.randint(0,w,(5000),dtype = int)
    for i in range(nums):
        if i%2 == 1:
            pic[rows[i],cols[i]] = (255,255,255)#奇数则产生白色
        else:
            pic[rows[i],cols[i]] = (0,0,0) #偶数则产生黑椒盐色
    #cv2.imshow("salt and pepper image", pic)
    #cv2.waitKey(0)

    return pic

addnoise(img_test)


array([[[202, 147, 110],
        [203, 148, 111],
        [203, 148, 111],
        ...,
        [188, 149, 135],
        [192, 154, 136],
        [196, 159, 139]],

       [[203, 148, 111],
        [203, 148, 111],
        [203, 148, 111],
        ...,
        [181, 144, 130],
        [191, 152, 137],
        [  0,   0,   0]],

       [[206, 148, 112],
        [206, 148, 112],
        [205, 147, 111],
        ...,
        [191, 153, 141],
        [199, 160, 145],
        [211, 173, 155]],

       ...,

       [[135, 137, 147],
        [134, 136, 146],
        [132, 134, 144],
        ...,
        [ 25,  14,  17],
        [ 25,  14,  17],
        [ 25,  14,  17]],

       [[132, 134, 144],
        [129, 131, 141],
        [125, 127, 137],
        ...,
        [ 28,  17,  20],
        [ 29,  18,  21],
        [ 29,  18,  21]],

       [[129, 131, 141],
        [123, 125, 135],
        [117, 119, 129],
        ...,
        [ 33,  22,  25],
        [ 34,  23,  26],
        [ 34,  23,  26]]

In [6]:
##均值滤波器
dst1 = cv2.blur(img_test,(3,3))
cv2.imshow("3*3", dst1)
cv2.waitKey()
cv2.destroyAllWindows()

In [4]:
##中值滤波器
dst2 = cv2.medianBlur(img_test, 3)
cv2.imshow("3*3", dst2)
cv2.waitKey()
cv2.destroyAllWindows()

In [5]:
##双边滤波器
dst3 = cv2.bilateralFilter(img_test, 15, 125, 200)
cv2.imshow("bilateral", dst3)
cv2.waitKey()
cv2.destroyAllWindows()

In [11]:
##Transforming it into a binary image as a set
def otsu(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    h, w = img.shape[:2]
    threshold_t = 0
    max_g = 0
    
    for t in range(255):
        front = img[img < t]
        back = img[img >= t]
        front_p = len(front) / (h * w)
        back_p = len(back) / (h * w)
        front_mean = np.mean(front) if len(front) > 0 else 0.
        back_mean = np.mean(back) if len(back) > 0 else 0.
        
        g = front_p * back_p * ((front_mean - back_mean)**2)
        if g > max_g:
            max_g = g
            threshold_t = t
    print(f"threshold = {threshold_t}")
    img[img < threshold_t] = 0
    img[img >= threshold_t] = 255
    return img

img_001 = otsu(img_test)

threshold = 126


In [12]:
cv2.imshow('img test',img_001)
cv2.waitKey()
cv2.destroyAllWindows()

In [17]:
##高斯滤波器
dst4 = cv2.GaussianBlur(img_001, (9, 9), 0, 0)
cv2.imshow("9*9", dst4)
cv2.waitKey()
cv2.destroyAllWindows()

In [13]:
#延展和打包
def extension(pic):
    width = []
    hight = []
    for index_row in range(len(pic)):
        if len(pic) > index_row and index_row > 0:
            hight.append(index_row)
    for index_value in range(len(pic[index_row])):
        if len(pic[0]) > index_value and index_value > 0:
            width.append(index_value)
    one = np.ones((1,len(pic[0])))
    one = one*255
    one = one.astype(int)
    #zero = zero.astype(str)
    pic = np.insert(pic, hight, one, axis=0)
    pic = np.insert(pic, width, 255 ,axis = 1)
    return pic

def package(pic):
    one = np.ones((1,len(pic[0])))
    one = one*255
    one = one.astype(int)
    #zero = zero.astype(str)
    pic = np.insert(pic, [0,len(pic)] , one, axis=0)
    pic = np.insert(pic, [0,len(pic)] , one, axis=0)
    pic = np.insert(pic,[0,len(pic[0])],255,axis = 1)
    pic = np.insert(pic,[0,len(pic[0])],255,axis = 1)
    return pic


In [14]:
VALUE = 0
SPACE = 255
#translate image into graph
def to_graph(pic):
    pic = pic.astype(int)
    extended_pic = extension(pic)
    new_pic = package(extended_pic)

    # a multidimensional array of equal size
    result = np.ones(extended_pic.shape)
    result = result*255
    result.astype(int)

    for index_row in range(len(new_pic)):
        for index_value in range(len(new_pic[index_row])):
            if index_row % 2 == 0 and index_value % 2 == 0:
                if (len(new_pic)-2 > index_row >= 2 and len(new_pic[0])-2 > index_value >=2) and (new_pic[index_row][index_value] == VALUE):
                    if new_pic[index_row][index_value+2] == VALUE and (
                        len(result) > index_row-2 >=0 and len(result[0]) > index_value and index_value-2 >=0):
                        result[index_row-2][index_value-2] = VALUE
                        result[index_row-2][index_value-1] = VALUE
                        result[index_row-2][index_value] = VALUE
                    if new_pic[index_row][index_value-2] == VALUE and (
                        len(result) > index_row-2 >=0 and len(result[0]) > index_value-2 and index_value-4 >=0):
                        result[index_row-2][index_value-2] = VALUE
                        result[index_row-2][index_value-3] = VALUE
                        result[index_row-2][index_value-4] = VALUE
                    if new_pic[index_row+2][index_value] == VALUE and (
                        len(result) > index_row and index_row-2 >=0 and len(result[0]) > index_value-2 >=0):
                        result[index_row-2][index_value-2] = VALUE
                        result[index_row-1][index_value-2] = VALUE
                        result[index_row][index_value-2] = VALUE
                    if new_pic[index_row-4][index_value] == VALUE and (
                        len(result) > index_row-2 and index_row-44 >=0 and len(result[0]) > index_value-2 >=0):
                        result[index_row-2][index_value-2] = VALUE
                        result[index_row-3][index_value-2] = VALUE
                        result[index_row-4][index_value-2] = VALUE
    return result
    


In [17]:
new_pic = to_graph(img_001)


cv2.imshow('gra',new_pic)
cv2.waitKey()
cv2.destroyAllWindows()

In [25]:
#opencv  erosion
kernel = np.ones(shape=[3,3],dtype=np.uint8)
img1 = cv2.erode(img_001,kernel=kernel)  # 腐蚀操作
cv2.imshow('img',img1)
cv2.waitKey()
cv2.destroyAllWindows()

In [26]:
kernel = np.ones(shape=[3,3],dtype=np.uint8)
img1 = cv2.dilate(img_001,kernel=kernel)  # dilation操作
cv2.imshow('img',img1)
cv2.waitKey()
cv2.destroyAllWindows()

In [38]:
#opencv  dilation
kernel = np.ones(shape=[3,3],dtype=np.uint8)
img1 = cv2.dilate(img_001,kernel=kernel)  # 腐蚀操作
cv2.imshow('img',img1)
cv2.waitKey()
cv2.destroyAllWindows()

In [18]:

def vertex_dilation(pic,gra):
    if gra == 0:
        pic = to_graph(pic)
        pic = pic.astype(int)
        new_pic = package(pic)

    if gra ==1:
        pic = pic.astype(int)
        new_pic = package(pic)

    # a multidimensional array of equal size
    result = np.ones(pic.shape)
    result.astype(int)

    for index_row in range(len(new_pic)):
        for index_value in range(len(new_pic[index_row])):
            if index_row % 2 == 0 and index_value % 2 == 0:
                if (len(new_pic)-2 >= index_row >= 2 and len(new_pic[0])-2 >= index_value >=2) and (new_pic[index_row][index_value+1] == VALUE
                    or new_pic[index_row][index_value-1] == VALUE
                    or new_pic[index_row+1][index_value] == VALUE
                    or new_pic[index_row-1][index_value] == VALUE):
                        if len(result) >= index_row-2 >= 0 and len(result[0]) >= index_value-2 >= 0:
                            result[index_row-2][index_value-2] = VALUE

    return result




def vertex_erosion(pic, gra):

    if gra == 0:
        pic = to_graph(pic)
        pic = pic.astype(int)
        new_pic = package(pic)

    if gra ==1:
        pic = pic.astype(int)
        new_pic = package(pic)

    # a multidimensional array of equal size
    result = np.ones(pic.shape)
    result.astype(int)

    for index_row in range(len(new_pic)):
        for index_value in range(len(new_pic[index_row])):
            if index_row % 2 == 0 and index_value % 2 == 0:
                if (len(new_pic)-2 >= index_row >= 2 and len(new_pic[0])-2 >= index_value >=2) and (new_pic[index_row][index_value+1] == VALUE
                    and new_pic[index_row][index_value-1] == VALUE
                    and new_pic[index_row+1][index_value] == VALUE
                    and new_pic[index_row-1][index_value] == VALUE):
                        if len(result) >= index_row-2 >= 0 and len(result[0]) >= index_value-2 >= 0:
                            result[index_row-2][index_value-2] = VALUE

    return result

In [19]:
def edge_dilation(pic, gra):

    if gra == 0:
        pic = to_graph(pic)
        pic = pic.astype(int)
        new_pic = package(pic)

    if gra ==1:
        pic = pic.astype(int)
        new_pic = package(pic)
    # a multidimensional array of equal size
    result = np.ones(pic.shape)
    result.astype(int)

    for index_row in range(len(new_pic)):
        for index_value in range(len(new_pic[index_row])):
            if (index_row % 2 == 0 and index_value % 2 == 1 
            and len(new_pic)-2 > index_row >= 2 and len(new_pic[0])-2 > index_value >=2):
                if (new_pic[index_row][index_value-1] == VALUE 
                or new_pic[index_row][index_value+1] == VALUE ):
                    if len(result) > index_row-2 >= 0 and len(result[0]) > index_value-2 >= 0:
                            result[index_row-2][index_value-2] = VALUE

            if (index_row % 2 == 1 and index_value % 2 == 0
                and len(new_pic)-2 > index_row >= 2 and len(new_pic[0])-2 > index_value >=2):
                if (new_pic[index_row-1][index_value] == VALUE 
                or new_pic[index_row+1][index_value] == VALUE ):
                    if len(result) > index_row-2 >= 0 and len(result[0]) > index_value-2 >= 0:
                            result[index_row-2][index_value-2] = VALUE          

    return result

def edge_erosion(pic, gra):

    if gra == 0:
        pic = to_graph(pic)
        pic = pic.astype(int)
        new_pic = package(pic)

    if gra ==1:
        pic = pic.astype(int)
        new_pic = package(pic)

    # a multidimensional array of equal size
    result = np.ones(pic.shape)
    result.astype(int)

    for index_row in range(len(new_pic)):
        for index_value in range(len(new_pic[index_row])):
            if (index_row % 2 == 0 and index_value % 2 == 1 
            and len(new_pic) > index_row > 2 and len(new_pic[0]) > index_value >=2):
                if (new_pic[index_row][index_value-1] == VALUE 
                and new_pic[index_row][index_value+1] == VALUE ):
                    if len(result) > index_row-2 >= 0 and len(result[0]) > index_value-2 >= 0:
                            result[index_row-2][index_value-2] = VALUE

            if (index_row % 2 == 1 and index_value % 2 == 0
                and len(new_pic)-2 > index_row >= 2 and len(new_pic[0])-2 > index_value >=2):
                if (new_pic[index_row-1][index_value] == VALUE 
                and new_pic[index_row+1][index_value] == VALUE ):
                    if len(result) > index_row-2 >= 0 and len(result[0]) > index_value-2 >= 0:
                            result[index_row-2][index_value-2] = VALUE          

    return result


In [20]:
def graph_dilation(pic, gra):

    #vertex
    pic_v = edge_dilation(pic, gra)
    pic_v = vertex_dilation(pic_v, 1)
    #edge
    pic_e = vertex_dilation(pic, gra)
    pic_e = edge_dilation(pic_e, 1)
    
    # a multidimensional array of equal size
    result = np.ones(pic_e.shape)
    result.astype(int)

    for index_row in range(len(result)):
        for index_value in range(len(result[index_row])):
            if pic_v[index_row][index_value] == VALUE or pic_e[index_row][index_value] == VALUE:
                result[index_row][index_value] = VALUE

    return result

def graph_eronsion(pic, gra):
    #vertex
    pic_v = edge_erosion(pic, gra)
    pic_v = vertex_erosion(pic_v, 1)
    #edge
    pic_e = vertex_erosion(pic, gra)
    pic_e = edge_erosion(pic_e, 1)
    
    # a multidimensional array of equal size
    result = np.ones(pic_e.shape)
    result.astype(int)

    for index_row in range(len(result)):
        for index_value in range(len(result[index_row])):
            if pic_v[index_row][index_value] == VALUE or pic_e[index_row][index_value] == VALUE:
                result[index_row][index_value] = VALUE

    return result

In [31]:
tmp = edge_dilation(img_001,0)
cv2.imshow('img',tmp)
cv2.waitKey()
cv2.destroyAllWindows()

In [38]:
def half_opening(pic, gra):
    #vertex
    pic_v = edge_erosion(pic, gra)
    pic_v = vertex_dilation(pic_v, 1)
    #edge
    pic_e = vertex_erosion(pic, gra)
    pic_e = edge_dilation(pic_e, 1)
    
    # a multidimensional array of equal size
    result = np.ones(pic_e.shape)
    result.astype(int)

    for index_row in range(len(result)):
        for index_value in range(len(result[index_row])):
            if pic_v[index_row][index_value] == VALUE or pic_e[index_row][index_value] == VALUE:
                result[index_row][index_value] = VALUE

    return result

def half_closing(pic, gra):
    #vertex
    pic_v = edge_dilation(pic, gra)
    pic_v = vertex_erosion(pic_v, 1)
    #edge
    pic_e = vertex_dilation(pic, gra)
    pic_e = edge_erosion(pic_e, 1)
    
    # a multidimensional array of equal size
    result = np.ones(pic_e.shape)
    result.astype(int)

    for index_row in range(len(result)):
        for index_value in range(len(result[index_row])):
            if pic_v[index_row][index_value] == VALUE or pic_e[index_row][index_value] == VALUE:
                result[index_row][index_value] = VALUE

    return result

In [39]:
def lambda_opening(pic, Lambda , gra):

    quotient = Lambda/2
    remainder = Lambda%2

    for i in range(int(quotient)):
        pic_new = graph_eronsion(pic, gra)

    if remainder == 1 and int(quotient) != 0:
        pic_new = half_opening(pic_new, 1)
    if remainder == 1 and int(quotient) == 0:
        pic_new = half_opening(pic, 1)
    
    for i in range(int(quotient)):
        
        pic_new = graph_dilation(pic_new, 1)

    return pic_new

def lambda_closing(pic, Lambda, gra):
    
    
    quotient = Lambda/2
    remainder = Lambda%2
    for i in range(int(quotient)):
        pic_new = graph_dilation(pic, gra)

    if remainder == 1 and int(quotient) != 0:
        pic_new = half_closing(pic_new, 1)
    if remainder == 1 and int(quotient) == 0:
        pic_new = half_closing(pic, 1)
    
    for i in range(int(quotient)):
        pic_new = graph_eronsion(pic_new, 1)

    return pic_new
    

In [40]:
ddd = lambda_closing(img_001,4,0)
cv2.imshow('img',ddd)
cv2.waitKey()
cv2.destroyAllWindows()

In [31]:
## opening
kernel = np.ones(shape=[3,3],dtype=np.uint8)
img1 = cv2.morphologyEx(src=img_001,op=cv2.MORPH_OPEN,kernel=kernel,iterations=1)  # 腐蚀操作
cv2.imshow('img',img1)
cv2.waitKey()
cv2.destroyAllWindows()

In [32]:
## closing
kernel = np.ones(shape=[3,3],dtype=np.uint8)
img1 = cv2.morphologyEx(src=img_001,op=cv2.MORPH_CLOSE,kernel=kernel,iterations=1)  # 腐蚀操作
cv2.imshow('img',img1)
cv2.waitKey()
cv2.destroyAllWindows()

In [29]:
def graph_opening(pic, gra):
    pic_1 = graph_dilation(pic, gra)
    pic_2 = graph_eronsion(pic_1, 1)

    result = pic_2
    return result

def graph_closing(pic, gra):
    pic_1 = graph_eronsion(pic, gra)
    pic_2 = graph_dilation(pic_1, 1)

    result =  pic_2
    return result

In [41]:
ddd = graph_opening(img_001, 0)
#ddd = graph_closing(img_001, 0)
cv2.imshow('img',ddd)
cv2.waitKey()
cv2.destroyAllWindows()

In [31]:
def ASF(pic, Lambda, gra):
    
    pic_1 = lambda_closing(pic, Lambda, gra)
    result_t = lambda_opening(pic_1, Lambda, 1)
    Lambda = Lambda-1
    if Lambda == 0:
        return result_t
    if Lambda != 0:
        return result_t - ASF(result_t, Lambda, 1)

In [32]:
ddd = ASF(img_001,3,0)
cv2.imshow('img',ddd)
cv2.waitKey()
cv2.destroyAllWindows()